In [5]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import math
import librosa
import tensorflow as tf
from collections import Counter

In [6]:
def load_data(dataset_path):
    # open file
    with open(dataset_path, "r") as fp:
        data = json.load(fp)

    # convert lists in to numpy arrays
    X = np.array(data["mfcc"])
    y = np.array(data["labels"])

    return X, y

In [7]:
def prepare_datasets(test_size, valid_size):
    X, y = load_data("processed.json")

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=valid_size)

    X_train = X_train[..., np.newaxis]
    X_valid = X_valid[..., np.newaxis]
    X_test = X_test[..., np.newaxis]
    
    return X_train, X_valid, X_test, y_train, y_valid, y_test

In [8]:
X_train, X_valid, X_test, y_train, y_valid, y_test = prepare_datasets(0.25, 0.2)

In [9]:
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
model = keras.Sequential(
    [
        keras.layers.Input(shape=input_shape),
        keras.layers.Conv2D(32, (3, 3), activation="relu"),
        keras.layers.MaxPool2D((3,3), strides=(2,2), padding="same"),
        keras.layers.BatchNormalization(),
        
        keras.layers.Conv2D(32, (3, 3), activation="relu"),
        keras.layers.MaxPool2D((3,3), strides=(2,2), padding="same"),
        keras.layers.BatchNormalization(),
        
        keras.layers.Conv2D(32, (2, 2), activation="relu"),
        keras.layers.MaxPool2D((2,2), strides=(2,2), padding="same"),
        keras.layers.BatchNormalization(),
        
        keras.layers.Flatten(),
        
        keras.layers.Dense(units=64, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(units=10, activation="linear")
    ]
)
optimizer = keras.optimizers.Adam(0.0001)
model.compile(
    optimizer=optimizer,
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [10]:
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=32, epochs=30)

Epoch 1/30
188/188 [==============================] - 5s 25ms/step - loss: 2.4068 - accuracy: 0.2278 - val_loss: 1.9389 - val_accuracy: 0.3164
Epoch 2/30
188/188 [==============================] - 4s 23ms/step - loss: 1.8738 - accuracy: 0.3539 - val_loss: 1.6620 - val_accuracy: 0.4166
Epoch 3/30
188/188 [==============================] - 4s 23ms/step - loss: 1.6866 - accuracy: 0.3979 - val_loss: 1.4988 - val_accuracy: 0.4559
Epoch 4/30
188/188 [==============================] - 4s 23ms/step - loss: 1.5590 - accuracy: 0.4470 - val_loss: 1.4060 - val_accuracy: 0.4973
Epoch 5/30
188/188 [==============================] - 4s 23ms/step - loss: 1.4707 - accuracy: 0.4745 - val_loss: 1.3283 - val_accuracy: 0.5287
Epoch 6/30
188/188 [==============================] - 5s 24ms/step - loss: 1.4106 - accuracy: 0.4939 - val_loss: 1.2884 - val_accuracy: 0.5507
Epoch 7/30
188/188 [==============================] - 5s 24ms/step - loss: 1.3631 - accuracy: 0.5098 - val_loss: 1.2260 - val_accuracy: 0.5654

In [11]:
test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Accuracy: {test_accuracy}, Error: {test_error}")

79/79 [==============================] - 1s 8ms/step - loss: 0.8211 - accuracy: 0.7229
Accuracy: 0.7228674292564392, Error: 0.8210904002189636


In [12]:
model.save("weights/cnn_weights.keras")

In [13]:
# Function to extract MFCCs from audio file
def extract_mfccs_from_audio(
    file_path,
    segment_duration=3,
    n_mfcc=13,
    n_fft=2048,
    hop_length=512,
    sample_rate=22050,
):
    signal, sr = librosa.load(file_path, sr=sample_rate)

    # Calculate the number of samples per segment
    samples_per_segment = sample_rate * segment_duration
    expected_vector_length = math.ceil(samples_per_segment / hop_length)

    mfccs = []
    num_segments = int(len(signal) / samples_per_segment)

    for s in range(num_segments):
        start_sample = samples_per_segment * s
        finish_sample = start_sample + samples_per_segment

        if finish_sample > len(signal):
            break

        mfcc = librosa.feature.mfcc(
            y=signal[start_sample:finish_sample],
            sr=sr,
            n_fft=n_fft,
            n_mfcc=n_mfcc,
            hop_length=hop_length,
        )
        mfcc = mfcc.T

        if len(mfcc) == expected_vector_length:
            mfccs.append(mfcc.tolist())

    return np.array(mfccs)

In [14]:
# Load the weights
model.load_weights("weights/cnn_weights.keras")

In [15]:
mapping = [
    "blues",
    "classical",
    "country",
    "disco",
    "hiphop",
    "jazz",
    "metal",
    "pop",
    "reggae",
    "rock",
]

In [16]:
mfccs = extract_mfccs_from_audio("sample songs/master.mp3")
predictions = model.predict(mfccs)

probabilities = tf.nn.softmax(predictions, axis=-1)

predicted_classes = np.argmax(probabilities, axis=1)

class_counts = Counter(predicted_classes)
most_common_class = mapping[class_counts.most_common(1)[0][0]]

print(f"The predicted class for the song is: {most_common_class}")

6/6 [==============================] - 0s 6ms/step
The predicted class for the song is: metal
